In [25]:
#!pip install transformers datasets accelerate huggingface_hub

In [26]:
# !pip install bitsandbytes

In [27]:
import os
import json
from getpass import getpass
from huggingface_hub import login

In [28]:
token = getpass(prompt="enter huggingface token")
login(token=token)

enter huggingface token ········


In [29]:
dir_name = "data"

if os.path.isfile(dir_name + '/topics.json'):
    with open(dir_name + '/topics.json') as t:
        topics = json.load(t)

    topics = topics.keys()
    tfiles = [dir_name + f"""/{"-".join(t.lower().split(' '))}.json""" for t in topics]
    tfiles = [tf for tf in tfiles if os.path.isfile(tf)]

    data = []
    for tf in tfiles:
        with open(tf) as t:
            data.extend(json.load(t))

    print(f'there are {len(data)} rows in the dataset')

there are 180 rows in the dataset


In [30]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

In [31]:
dataset = load_dataset("json", data_files=tfiles)

In [32]:
dataset = dataset["train"].train_test_split(test_size=0.3, seed=42)

In [33]:
def preprocess(row):
    row['input'] = f'<question>\n{row["question"]}\n</question>\n\n<documents>\n{row["documents"]}\n</documents>\n\n<answer>'
    return row

In [34]:
dataset = dataset.map(preprocess).shuffle(seed=42)

In [35]:
print(dataset['train'][0]['input'])

<question>
What are the projected rare earth element requirements for achieving global net-zero emissions by 2050?
</question>

<documents>
Document 1: Rare earth elements play a crucial role in the development of green technologies that are essential for reducing carbon emissions. Neodymium and dysprosium are key components in the permanent magnets used in wind turbines, allowing for more efficient electricity generation without mechanical gearboxes. These magnets are also essential in electric vehicle motors, with each EV requiring approximately 1kg of rare earth materials. Additionally, rare earth elements like europium, terbium, and yttrium are used in energy-efficient LED lighting, reducing energy consumption compared to traditional lighting solutions.
---
Document 2: Demand-side economics for rare earths are driven by growing applications in clean energy, transportation electrification, and advanced electronics. The permanent magnet sector represents the largest value segment, co

In [36]:
from transformers import BitsAndBytesConfig

model_id = "meta-llama/Llama-3.2-1B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [37]:
# model = model.to(torch.device('cpu'))

In [38]:
tokenizer.pad_token = tokenizer.eos_token

In [39]:
def tokenize_function(row):
    prompt = row["input"]
    full_text = prompt + "\n" + f'{row["answer"]}\n</answer>'

    tokenized = tokenizer(
        full_text,
        truncation=True,
        max_length=1536,
        padding="max_length",
    )

    prompt_tokenized = tokenizer(
        prompt,
        truncation=True,
        max_length=1536,
        add_special_tokens=False  # Ensure no extra tokens are added
    )
    prompt_length = len(prompt_tokenized["input_ids"])

    # Copy the tokenized full text to create labels and mask out prompt tokens
    labels = tokenized["input_ids"].copy()
    labels[:prompt_length] = [-100] * prompt_length  # -100 tells PyTorch to ignore these tokens in loss computation

    tokenized["labels"] = labels
    return tokenized

In [40]:
tokenized_dataset = dataset.map(tokenize_function)

Map:   0%|          | 0/126 [00:00<?, ? examples/s]

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

In [41]:
from peft import LoraConfig, get_peft_model

# Define LoRA Configuration
lora_config = LoraConfig(
    r=4,               # Low-rank dimension
    lora_alpha=32,     # Scaling factor
    lora_dropout=0.1,  # Dropout for regularization
    bias="none",       # No bias training
    task_type="CAUSAL_LM"  # Task type: Causal Language Modeling
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 425,984 || all params: 1,236,240,384 || trainable%: 0.0345


In [42]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./llama-1b-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=30,
    logging_steps=10,
    save_strategy="epoch",
    report_to=None
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test']
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
10,7.698400
20,3.253800
30,0.713500
40,0.315900
50,0.296900
60,0.284700
70,0.260600
80,0.268800
90,0.248300
100,0.241100


TrainOutput(global_step=240, training_loss=0.6859187081456184, metrics={'train_runtime': 869.5274, 'train_samples_per_second': 4.347, 'train_steps_per_second': 0.276, 'total_flos': 3.391582485086208e+16, 'train_loss': 0.6859187081456184, 'epoch': 30.0})

In [44]:
#torch.cuda.empty_cache()

In [50]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./llama-1b-finetuned/checkpoint-224", device="cuda", max_length=1536)

In [51]:
response = pipe(
    """
<question>
How should we treat animals?
</question>

<documents>
Document 1: But it’s been thirty years and vegans are still less than 10% of the population.If you genuinely care about animal suffering, you have to admit that, and say, ‘what else can we do to ease animal suffering?’”After Singer’s talk, I began thinking through the consequences of his morality.A question occurred to me: “Should we also stop animals from eating each other?” I was sure others had made such arguments as *reductio ad absurdums* of vegetarianism, but I thought I might be the first to be genuinely interested in it from a moral perspective.
---
Document 2: They didn’t even pretend to make an argument; it was entirely mockery.Watching this, I couldn’t help but realize there was a powerful logical argument at the core of the animal rights groups: animals should be treated much the same way humans are — their lives should be respected, their pain minimized, etc.Make this one simple change to your system of morality and everything else falls into place.
---
Document 3: But I praise people for the good things they do and condemn them for the bad ones.”A final question raised the incrementalism versus revolutionism debate common to all left-wing social movements.Should one really worry about animal treatment when the animals were still going to be killed?Pinger said the answer was undoubtedly yes.
---
Document 4: They didn’t even pretend to make an argument; it was entirely mockery.Watching this, I couldn’t help but realize there was a powerful logical argument at the core of the animal rights groups: animals should be treated much the same way humans are — their lives should be respected, their pain minimized, etc.Make this one simple change to your system of morality and everything else falls into place.
---
Document 5: But I praise people for the good things they do and condemn them for the bad ones.”A final question raised the incrementalism versus revolutionism debate common to all left-wing social movements.Should one really worry about animal treatment when the animals were still going to be killed?Pinger said the answer was undoubtedly yes.
---
Document 6: But it’s been thirty years and vegans are still less than 10% of the population.If you genuinely care about animal suffering, you have to admit that, and say, ‘what else can we do to ease animal suffering?’”After Singer’s talk, I began thinking through the consequences of his morality.A question occurred to me: “Should we also stop animals from eating each other?” I was sure others had made such arguments as *reductio ad absurdums* of vegetarianism, but I thought I might be the first to be genuinely interested in it from a moral perspective.
</documents>

<answer>
""".strip()
)
print(response[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<question>
How should we treat animals?
</question>

<documents>
Document 1: But it’s been thirty years and vegans are still less than 10% of the population.If you genuinely care about animal suffering, you have to admit that, and say, ‘what else can we do to ease animal suffering?’”After Singer’s talk, I began thinking through the consequences of his morality.A question occurred to me: “Should we also stop animals from eating each other?” I was sure others had made such arguments as *reductio ad absurdums* of vegetarianism, but I thought I might be the first to be genuinely interested in it from a moral perspective.
---
Document 2: They didn’t even pretend to make an argument; it was entirely mockery.Watching this, I couldn’t help but realize there was a powerful logical argument at the core of the animal rights groups: animals should be treated much the same way humans are — their lives should be respected, their pain minimized, etc.Make this one simple change to your system of moral